### Model Training Setup

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
cd ..

c:\Users\Matheus\Documents\Git\knowledge-graph-completion


### Load data

In [3]:
import pandas as pd
from src.utils import load_fb15k237

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("max_colwidth", None)

PATH_FB15k237 = "data/datasets_knowledge_embedding/FB15k-237"

train, valid, test, entity2wikidata = load_fb15k237(PATH_FB15k237)
processed_data = pd.read_csv(PATH_FB15k237 + "/processed_data_v2.csv")

### Load the model

In [6]:
from transformers import (
    BartForConditionalGeneration,
    BartTokenizer,
    DataCollatorForSeq2Seq,
)

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Select model
# MODEL = "facebook/bart-large"
# MODEL = "facebook/bart-base"
MODEL = "lucadiliello/bart-small"

# Load model and tokenizer
model = BartForConditionalGeneration.from_pretrained(MODEL).cuda().float().to(device)
tokenizer = BartTokenizer.from_pretrained(MODEL)

DEV_BATCH = 100
MAX_LENGTH = 50  # model.config.d_model
BATCH_SIZE = 1

dev = True

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


### Masking data

In [7]:
processed_data["data_input"] = (
    processed_data["demonstration_input"] + "%s." % tokenizer.mask_token
)
processed_data["data_label"] = (
    processed_data["demonstration_input"] + processed_data["tail_text"] + "."
)

if dev:
    if DEV_BATCH == -1:
        pass
    else:
        processed_data = processed_data.head(DEV_BATCH)

In [8]:
from torch.utils.data import Dataset
import copy
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding
from tqdm.auto import tqdm


random.seed(42)


class DatasetKGC(Dataset):
    def __init__(self, data):
        self.data = data
        self.data["input_ids"] = self.data["input_ids"]
        self.data["labels"] = self.data["labels"]
        self.num_rows = self.data["input_ids"].shape[0]

    def __len__(self):
        return self.num_rows

    def __getitem__(self, idx):
        _input = self.data["input_ids"][idx].squeeze(0)
        label = self.data["labels"][idx].squeeze(0)

        return {"input_ids": _input, "labels": label}


def encode_data(data):
    # Codifica as strings de entrada e rótulos como sequências de tokens BART
    encoded_input = tokenizer(
        list(data["data_input"]),
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        max_length=MAX_LENGTH,
        add_special_tokens=True,
        return_attention_mask=False,
    )
    encoded_label = tokenizer(
        list(data["data_label"]),
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        max_length=MAX_LENGTH,
        add_special_tokens=True,
        return_attention_mask=False,
    )

    # Cria uma lista de exemplos
    examples = []
    for i in range(len(data)):
        input_ids = encoded_input["input_ids"][i]
        labels = encoded_label["input_ids"][i]
        examples.append({"input_ids": input_ids, "labels": labels})

    # Cria um objeto DataCollatorForLanguageModeling
    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer, padding="max_length", max_length=MAX_LENGTH
    )

    # Prepara os dados de treinamento
    prepared_data = data_collator(examples)

    return prepared_data


def train_valid_split(data):
    train, valid = train_test_split(data, test_size=0.2, random_state=42)
    return encode_data(train), encode_data(valid)


def generate_train_valid_dataloader(data):
    train, valid = train_valid_split(data)

    train_loader = DataLoader(DatasetKGC(train), batch_size=BATCH_SIZE, shuffle=False)

    valid_loader = DataLoader(DatasetKGC(valid), batch_size=BATCH_SIZE, shuffle=False)

    return train_loader, valid_loader


def generate_train_valid_dataset(data):
    train, valid = train_valid_split(data)

    train_loader = DatasetKGC(train)

    valid_loader = DatasetKGC(valid)

    return train_loader, valid_loader


def _decode(tokens, tokenizer=tokenizer, batch=True):
    # Remove padding tokens and decode
    # tokens = tokens[tokens != -100]
    if batch:
        return tokenizer.batch_decode(tokens, skip_special_tokens=True)

    return tokenizer.decode(tokens, skip_special_tokens=True)


def _decode_error(tokens_i, tokens_j, tokenizer=tokenizer):
    """ """
    # Remove padding tokens
    text_i = _decode(tokens_i)
    text_j = _decode(tokens_j)

    # print(text_i)
    # print(text_j)

    error = 0

    len_i = len(tokens_i)
    len_j = len(tokens_j)
    total_tokens = max(len_i, len_j)

    diff_shape = abs(len_i - len_j)

    error += diff_shape

    for i in range(min(len_i, len_j)):
        if text_i[i] != text_j[i]:
            error += 1

    return (error, total_tokens, error / total_tokens)


def _decode_error_epoch(model, valid_dataset, debug=False):
    error_tokens = 0
    total_tokens = 0

    with torch.no_grad():
        for _input, label in tqdm(valid_dataset, desc="Analyzing decoder error"):
            model_output = model.generate(_input.to(device), max_length=MAX_LENGTH)

            for i in range(len(model_output)):
                a, b, percentage = _decode_error(label[i], model_output[i].to("cpu"))
                error_tokens += a
                total_tokens += b

        if debug:
            print("Total tokens analyzed: %d" % total_tokens)
            print("Total erroneous tokens predicted: %d" % error_tokens)
            print("Percentage of error: %.3f%%" % ((error_tokens / total_tokens) * 100))

        return error_tokens, total_tokens, error_tokens / total_tokens

In [ ]:
# Main pipeline
# train_loader, valid_loader = generate_train_valid_dataloader(processed_data)

# Overfitting test
# train_loader = DataLoader(
#     DatasetKGC(encode_data(processed_data)), batch_size=BATCH_SIZE, shuffle=False
# )
# valid_loader = train_loader

train_ds, valid_ds = generate_train_valid_dataset(processed_data)

In [ ]:
# We have a small error because the mask token is not filled at this point.
# sample_feature = valid_features[0]
# sample_label = valid_labels[0]
# sample_output = model.generate(
#     sample_feature.reshape(1, -1).to(device), max_length=MAX_LENGTH
# )

# print(_decode(sample_feature, batch=False))
# print(_decode(sample_label, batch=False))
# print(_decode(sample_output[0], batch=False))

### Training Model

In [ ]:
from transformers import AdamW, get_scheduler

epochs = 60
epoch_accuracy_frequency = 30  # if epoch % accuracy -> compute
loss_epoch = []
lr = 1e-5
cross = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
# # Train

# from tqdm.auto import tqdm

# pbar = tqdm(range(1, epochs + 1), desc="Epochs")
# decoder_error = []

# decoder_error.append(_decode_error_epoch(model, valid_loader, debug=True))

# for epoch in pbar:
#     epoch_loss = 0

#     pbar.set_description("Epoch %s" % epoch)
#     pbar.refresh()

#     for _input, label in train_loader:
#         model.zero_grad()

#         _dt = model(_input.to(device), labels=label.to(device), return_dict=True)

#         # _dt_label = model(label.to(device), return_dict=True)

#         # logits_input = _dt.logits
#         # logits_label = _dt_label.logits

#         # loss = cross(
#         #     logits_input.view(-1, logits_input.size(-1)).softmax(dim=-1),
#         #     logits_label.view(-1, logits_input.size(-1)),
#         # )

#         loss = _dt.loss

#         epoch_loss += loss.item()

#         loss.backward()
#         optimizer.step()

#     if epoch % epoch_accuracy_frequency == 0:
#         decoder_error.append(_decode_error_epoch(model, valid_loader, debug=True))

#     pbar.set_postfix(loss=epoch_loss)
#     loss_epoch.append(epoch_loss)

#     torch.cuda.empty_cache()

In [ ]:
loss_values = []


def compute_metrics(eval_pred):
    global loss_values
    loss = float(eval_pred["loss"])
    loss_values.append(eval_pred)

    return {"loss": loss}

In [ ]:
from transformers import Trainer
from transformers import TrainingArguments

params = {
    # Dir
    "output_dir": "model/model_bart",
    # Batch
    "per_device_train_batch_size": 2,
    "per_device_eval_batch_size": 2,
    # Learning rate
    "learning_rate": 5e-5,
    "seed": 42,
    # Epochs
    "num_train_epochs": 50,
    # Logging
    "logging_dir": "model/logs",
    "logging_strategy": "epoch",
    "logging_steps": 10,
    # Evaluation
    "evaluation_strategy": "epoch",
    "eval_steps": 10,
    # Checkpoint
    "save_strategy": "epoch",
    "save_steps": 10,
    "save_total_limit": 2,
    "report_to": "tensorboard",
    "ddp_find_unused_parameters": False,
    "warmup_steps": 2,
}

training_args = TrainingArguments(
    # Dir
    output_dir=params["output_dir"],
    # Batch
    per_device_train_batch_size=params["per_device_train_batch_size"],
    per_device_eval_batch_size=params["per_device_eval_batch_size"],
    # Learning Rate
    learning_rate=params["learning_rate"],
    seed=params["seed"],
    # Epoch
    num_train_epochs=params["num_train_epochs"],
    # logging
    logging_dir=params["logging_dir"],
    logging_strategy=params["logging_strategy"],
    logging_steps=params["logging_steps"],
    # Evaluation
    # evaluation_strategy=params["evaluation_strategy"],
    # eval_steps=params["eval_steps"],
    # Checkpoint
    save_strategy=params["save_strategy"],
    save_steps=params["save_steps"],
    save_total_limit=params["save_total_limit"],
    # pretraining
    ddp_find_unused_parameters=params["ddp_find_unused_parameters"],
    warmup_steps=params["warmup_steps"],
    fp16=True,
    fp16_full_eval=True,
    # test
    eval_accumulation_steps=1,
)


data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer, padding="max_length", max_length=MAX_LENGTH
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [34]:
trainer.train()

Saving model checkpoint to model/model_bart\checkpoint-200
Configuration saved in model/model_bart\checkpoint-200\config.json


{'loss': 0.0035, 'learning_rate': 4.507007007007007e-05, 'epoch': 5.0}


Model weights saved in model/model_bart\checkpoint-200\pytorch_model.bin
tokenizer config file saved in model/model_bart\checkpoint-200\tokenizer_config.json
Special tokens file saved in model/model_bart\checkpoint-200\special_tokens_map.json
Deleting older checkpoint [model\model_bart\checkpoint-120] due to args.save_total_limit
Saving model checkpoint to model/model_bart\checkpoint-240
Configuration saved in model/model_bart\checkpoint-240\config.json


{'loss': 0.0028, 'learning_rate': 4.406906906906907e-05, 'epoch': 6.0}


Model weights saved in model/model_bart\checkpoint-240\pytorch_model.bin
tokenizer config file saved in model/model_bart\checkpoint-240\tokenizer_config.json
Special tokens file saved in model/model_bart\checkpoint-240\special_tokens_map.json
Deleting older checkpoint [model\model_bart\checkpoint-160] due to args.save_total_limit
Saving model checkpoint to model/model_bart\checkpoint-280
Configuration saved in model/model_bart\checkpoint-280\config.json


...

Model weights saved in model/model_bart\checkpoint-1960\pytorch_model.bin
tokenizer config file saved in model/model_bart\checkpoint-1960\tokenizer_config.json
Special tokens file saved in model/model_bart\checkpoint-1960\special_tokens_map.json
Deleting older checkpoint [model\model_bart\checkpoint-1880] due to args.save_total_limit
Saving model checkpoint to model/model_bart\checkpoint-2000
Configuration saved in model/model_bart\checkpoint-2000\config.json


{'loss': 0.0001, 'learning_rate': 2.5025025025025025e-08, 'epoch': 50.0}


Model weights saved in model/model_bart\checkpoint-2000\pytorch_model.bin
tokenizer config file saved in model/model_bart\checkpoint-2000\tokenizer_config.json
Special tokens file saved in model/model_bart\checkpoint-2000\special_tokens_map.json
Deleting older checkpoint [model\model_bart\checkpoint-1920] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 320.2615, 'train_samples_per_second': 12.49, 'train_steps_per_second': 6.245, 'train_loss': 0.001519653731258586, 'epoch': 50.0}


TrainOutput(global_step=2000, training_loss=0.001519653731258586, metrics={'train_runtime': 320.2615, 'train_samples_per_second': 12.49, 'train_steps_per_second': 6.245, 'train_loss': 0.001519653731258586, 'epoch': 50.0})

In [67]:
data = train_ds[0]

In [68]:
data["input_ids"].shape

torch.Size([512])

In [69]:
with torch.no_grad():
    print("Sample input: ")
    print(tokenizer.decode(data["input_ids"]).replace("<pad>", ""), "\n")

    print("Expected output: ")
    print(tokenizer.decode(data["labels"], skip_special_tokens=True), "\n")

    print("Model Output: ")
    print(
        tokenizer.decode(
            model.generate(
                data["input_ids"].to(device).reshape(1, -1), max_length=MAX_LENGTH
            )[0]
        )
    )

Sample input: 
<s>Jenna Ushkowitz has award winner of Josh Sussman. Paul Dini has award winner of Adam Horowitz. Don Cheadle has award winner of<mask>.</s> 

Expected output: 
Jenna Ushkowitz has award winner of Josh Sussman. Paul Dini has award winner of Adam Horowitz. Don Cheadle has award winner of Larenz Tate. 

Model Output: 
</s><s>Jenna Ushkowitz has award winner of Josh Sussman. Paul Dini has award winners of Adam Horowitz. Don Cheadle has award loser of Larenz Tate.</s>


In [45]:
model.save_pretrained("model/saves")

Configuration saved in model/saves\config.json
Model weights saved in model/saves\pytorch_model.bin


### Train plots

In [ ]:
import matplotlib.pyplot as plt

plt.title("Transformer Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
# plt.xticks(list(range(epochs + 1)))
plt.plot(loss_epoch)
print(loss_epoch[-1])

In [ ]:
import matplotlib.pyplot as plt

plt.title("Decoder Error")
plt.xlabel("Epoch")
plt.ylabel("token error rate")
plt.xticks(
    list(map(lambda i: i * epoch_accuracy_frequency, list(range(len(decoder_error)))))
)
plt.plot(
    list(map(lambda i: i * epoch_accuracy_frequency, list(range(len(decoder_error))))),
    [i[2] for i in decoder_error],
)